In [3]:
import pandas as pd
import requests
import os

excel_file = "/home/workdir/document_classification/data/DataSet.xlsx"
excel_sheets = pd.read_excel(excel_file, sheet_name=None)
for sheet_name, df in excel_sheets.items():
    print(sheet_name)
    globals()[sheet_name] = df

train_data
test_data


In [3]:
train_data.shape

(2570, 2)

In [4]:
test_data.shape

(400, 2)

In [6]:
failed_urls = []
failed_ids = []
timeout_urls = []
timeout_ids = []

def download_pdf(url, pdf_dir, filename, idx):
    try:
        response = requests.get(url, timeout=90)
        response.raise_for_status()

        filepath = os.path.join(pdf_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        return True
    except requests.exceptions.Timeout:
        print(f"Skipping document {idx} due to high wait time (>120 seconds)")
        timeout_urls.append(url)
        timeout_ids.append(idx)
        return False
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")
        failed_urls.append(url)
        failed_ids.append(idx)
        return False


In [ ]:

pdf_dir = "/home/jovyan/workdir/document_classification/data/train_pdfs"
os.makedirs(pdf_dir, exist_ok=True)
for idx, url in enumerate(train_data["datasheet_link"]):
    if isinstance(url, str) and url.strip():
        filename = f"doc_{idx}.pdf"
        success = download_pdf(url, pdf_dir, filename, idx)
        if success:
            pass
            # print(f"Successfully downloaded: {filename}")
        else:
            print(f"Failed to download document {idx}")

print(len(failed_urls))

Error downloading https://lfillumination.com/files/specsheets/EF411B.PDF: 404 Client Error: Not Found for url: https://lfillumination.com/files/specsheets/EF411B.PDF
Failed to download document 1
Error downloading https://www.acuitybrands.com/api/products/getasset/holophane/1649332/4408a251-09be-4b61-813f-177b3ebcf10c/holophane-care222-hldmpc-cylinder-pendant-with-cord.pdf?abl_version=01%2f12%2f2023+16%3a54%3a24&DOC_Type=SPEC_SHEET: 404 Client Error: Not Found for url: https://www.acuitybrands.com/api/products/getasset/holophane/1649332/4408a251-09be-4b61-813f-177b3ebcf10c/holophane-care222-hldmpc-cylinder-pendant-with-cord.pdf?abl_version=01%2F12%2F2023+16%3A54%3A24&DOC_Type=SPEC_SHEET
Failed to download document 5
Error downloading https://beghelliusa.com/wp-content/uploads/2022/05/testa_spec.pdf: 404 Client Error: Not Found for url: https://beghelliusa.com/wp-content/uploads/2023/08/testa_spec.pdf
Failed to download document 20
Error downloading https://beghelliusa.com/wp-content/up

In [ ]:

timeout_df = pd.DataFrame({"url": timeout_urls, "id": timeout_ids, "label": "timeout"})

failed_df = pd.DataFrame({"url": failed_urls, "id": failed_ids, "label": "failed"})

all_errors_df = pd.concat([timeout_df, failed_df])

all_errors_df.to_csv(
    "/home/jovyan/workdir/document_classification/artifacts/test_download_errors.csv",
    index=False,
)
